In [ ]:
from IPython.core.display import HTML
HTML(open('../style.css').read())

In [ ]:
%load_ext nb_mypy

# A$^*$ First Search

In [ ]:
from typing import TypeVar, Callable

In [ ]:
State     = TypeVar('State')
NxtStFct  = Callable[[State], set[State]] 
Heuristic = Callable[[State, State], float]

The module [heapq](https://docs.python.org/3.7/library/heapq.html) provides 
[priority queues](https://en.wikipedia.org/wiki/Priority_queue) 
that are implemented as [heaps](https://en.wikipedia.org/wiki/Heap_(data_structure)).
Technically, these heaps are just lists.  In order to use them as priority queues, the entries of these lists will be pairs of the form $(p, o)$, where $p$ is the priority of the object $o$.  Usually, the priorities are numbers 
and, contra-intuitively, high priorities correspond to <b>small</b> numbers, that is $(p_1, o_1)$ has a higher priority than $(p_2, o_2)$ iff $p_1 < p_2$.

We need only two functions from the module `heapq`:
- Given a heap $H$, the function $\texttt{heapq.heappop}(H)$ removes the pair
  from H that has the highest priority.  This pair is also returned.
- Given a heap $H$, the function $\texttt{heapq.heappush}\bigl(H, (p, o)\bigr)$ 
  pushes the pair $(p, o)$ onto the heap $H$.  This method does not return a 
  value.  Instead, the heap $H$ is changed in place.

In [ ]:
import heapq

The function `search` takes three arguments to solve a *search problem*:
- `start` is the start state of the search problem,
- `goal` is the goal state, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
  For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.
- `heuristic` is a function that takes two states as arguments.  It returns an estimate of the 
  length of the shortest path between these states.
If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$

The variable `PrioQueue` that is used in the implementation contains pairs of the form
$$ \bigl(\texttt{len}(\texttt{Path}) + \texttt{heuristic}(\texttt{state},\; \texttt{goal}), \texttt{Path}\bigr), $$
where `Path` is a path from `start` to `state` and $\texttt{heuristic}(\texttt{state}, \texttt{goal})$
is an estimate of the distance from `state` to `goal`.  The idea is to always extend the most promising `Path`, i.e. to extend the `Path` whose completed version would be shortest.

We can modify *$\mathrm{A}^*$ search* into *weighted $\mathrm{A}^*$ search* by changing the priority 
$$ \texttt{len}(\texttt{Path}) + \texttt{heuristic}(\texttt{state}) 
   \qquad \mbox{into} \qquad
   \texttt{len}(\texttt{Path}) + \texttt{weight} \cdot \texttt{heuristic}(\texttt{state}).
$$
Here `weight` is a real number greater or equal than $1.0$.  This modification
can speed up the solution and decrease the memory usage.  However, the resulting algorithm does not necessarily find the shortest path. 

In [ ]:
weight = 1.0

In [ ]:
def search(start: State, goal: State, next_states: NxtStFct, heuristic: Heuristic) -> list[State] | None:
    Visited: set[State] = set()
    PrioQueue = [ (heuristic(start, goal), [start]) ]
    while PrioQueue:
        _, Path = heapq.heappop(PrioQueue)
        state   = Path[-1]
        if state in Visited:
            continue
        if state == goal:
            print(f'Number of nodes visited: {len(Visited)}')
            return Path
        for ns in next_states(state):           
            if ns not in Visited:
                prio = len(Path) + weight * heuristic(ns, goal)
                heapq.heappush(PrioQueue, (prio, Path + [ns]))
        Visited.add(state)
    return None

In [ ]:
%run 03-Sliding-Puzzle.ipynb

In [ ]:
%load_ext memory_profiler

In [ ]:
%%time
%memit Path = search(start, goal, next_states, manhattan)
print("The path has the length ", len(Path)-1)

In [ ]:
animation(Path) # type: ignore

Solving the 15-puzzle can be done in less than a second.

In [ ]:
%%time
%memit Path = search(start2, goal2, next_states, manhattan)
print(f'The path has a length of {len(Path)-1} steps.')

In [ ]:
animation(Path) # type: ignore